# Fundamentals of Data Analysis
## Tasks

These are my solutions to the Tasks assessment for this module. The author is Brian Doheny.

### Task 1

Write a Python function called counts that takes a list as input and returns a dictionary of unique items in the list as keys and the number of times each item appears as values. So, the input ['A', 'A', 'B', 'C', 'A'] should have output {'A': 3, 'B': 1, 'C': 1} . Your code should not depend on any module from the standard library or otherwise. You should research the task first and include a description with references of your algorithm in the notebook.

### Pseudocode

<p>for item in list:</p>
    <p>if item in dictionary.keys:</p>
        <p>item.value +1</p>
    <p>else:</p> 
        <p>dictionary.append(key=item, value=1)</p>


### References
<p>Example</p>
<p>[1] Real Python; How to Iterate Through a Dictionary in Python; https://realpython.com/iterate-through-dictionary-python/</p>
<p>[2] Career Karma; Python Add to Dictionary: A Guide; https://careerkarma.com/blog/python-add-to-dictionary/</p>
<p>[3] Geeks for Geeks; Python | Get specific keys' values; https://www.geeksforgeeks.org/python-get-specific-keys-values/?ref=rp

In [14]:
#First attempt at the task

test = ['A', 'A', 'B', 'C', 'A']

results = {}
for item in test:
    if item in results.keys():
        results[item] += 1
    else:
        results[item] = 1

print(results)
        

{'A': 3, 'B': 1, 'C': 1}
